In [1]:
from __future__ import print_function, division
import uproot
import numpy as np
#Make sure to install both old awkward0 and new awkward1(referred to now as awkward)
import awkward1 as ak
import awkward0 as ak0
from coffea.nanoevents import NanoAODSchema,NanoEventsFactory
from uproot3_methods import TLorentzVectorArray
import uproot3_methods
import numpy as np
import coffea.hist as hist
import matplotlib.pyplot as plt
import awkward
class HackSchema(NanoAODSchema):
    def __init__(self, base_form):
        base_form["contents"].pop("Muon_fsrPhotonIdx", None)
        base_form["contents"].pop("Electron_photonIdx", None)
        super().__init__(base_form)
def m3_recon(tree):
    comb= ak.combinations(tree,n=3,axis=1,fields=['j1','j2','j3'])
    trijets= comb.j1+comb.j2+comb.j3
    recon =ak.max(trijets,axis=1)
    reconfinal=np.sqrt(recon.t*recon.t-recon.x*recon.x-recon.y*recon.y-recon.z*recon.z)
    list1= ak.to_numpy(reconfinal)
    return list1
files ="TTbarPowheg_Semilept_Skim_NanoAOD_1of21.root"
import coffea.processor as processor
from pprint import pprint
file=uproot.open(files)
nEvents=file['hEvents'].values[0]+file['hEvents'].values[2]
from pprint import pprint

C:\Users\zshel\anaconda3\envs\top_tag1\lib\site-packages\awkward0\__init__.py:23: FutureWarning: Consider switching from 'awkward0' to 'awkward', since the new interface became the default in 2020.

    pip install -U awkward

In Python:

    >>> import awkward as ak
    >>> new_style_array = ak.from_awkward0(old_style_array)
    >>> old_style_array = ak.to_awkward0(new_style_array)

  FutureWarning


In [2]:
class HackSchema(NanoAODSchema):
    def __init__(self, base_form):
        base_form["contents"].pop("Muon_fsrPhotonIdx", None)
        base_form["contents"].pop("Electron_photonIdx", None)
        super().__init__(base_form)
def m3_recon(tree):
    #print(tree[1])
    comb= ak.combinations(tree,n=3,axis=1,fields=['j1','j2','j3'])
    #print(comb[1])
    trijets= comb.j1+comb.j2+comb.j3
    recon =ak.max(trijets,axis=1)
    #print(recon[1])
    reconfinal=np.sqrt(recon.t*recon.t-recon.x*recon.x-recon.y*recon.y-recon.z*recon.z)
    list1= ak.to_numpy(reconfinal)
    return list1
files ="TTbarPowheg_Semilept_Skim_NanoAOD_1of21.root"
class JetPreProcess(processor.ProcessorABC):
    def __init__(self):

        #Declare an axis for the dataset
        dataset_axis = hist.Cat("dataset","Dataset")
        
        #Declare an axis for the muon pt
        pt_axis = hist.Bin("pt","$p_{T}$ [GeV]", 50, 0, 200)
        mass_axis= hist.Bin("mass","Mass [GeV]",100,0,1000)

        
        #Utilizing Column Accumulators, the structure of data can be retained utilizing the pad_none function. during conversion to Numpy arrays
        #None values are converted to zeros.
        self._accumulator = processor.dict_accumulator({
            'pre-cuts_M3': hist.Hist("Counts", dataset_axis, mass_axis),
            'post-cuts_M3':hist.Hist("Counts", dataset_axis, mass_axis),
            'pre-cuts_pt': hist.Hist("Counts", dataset_axis, pt_axis),
            'post-cuts_pt':hist.Hist("Counts", dataset_axis, pt_axis),
            'jet_phi':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'jet_eta':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'jet_pt':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'jet_mass':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'genjet_id':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'pdgId':processor.column_accumulator(np.ndarray(shape=(1,8)))
        }
        )

    @property
    def accumulator(self):
        return self._accumulator

    def process(self, events):
        
        output = self.accumulator.identity()
        dataset = events.metadata['dataset']
        #####
        #Tight Lepton selection
        #####
        tight_muons = events.Muon
        muonsel=ak.num(tight_muons[((tight_muons.pt>30)&(abs(tight_muons.eta)<2.4))],axis=1)==1
        tight_electrons= events.Electron
        electronsel=ak.num(tight_electrons[((tight_electrons.pt>35)&(abs(tight_electrons.eta)<2.4))],axis=1)==1
        leptonsel=(muonsel|electronsel)
        
        ######
        #Selects Jets with Jet cuts AND at least one b-tagged jet
        ######
        tight_jets=events.Jet
        jetSel = ak.num(tight_jets[((tight_jets.pt>30)&(tight_jets.eta<2.4)&(tight_jets.eta>-2.4))],axis=1)>=3
        jetSelection=(jetSel&(ak.num(tight_jets.btagCSVV2>.4184)>=1))
        
        ######
        #Leading Jet Delta_r cuts for preprocessing
        #JetSelection and Lepton Selection combined create final "Pre-processed Data"
        
        
        f_events=events[(jetSelection&leptonsel)]
        jet_pt=(processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(f_events.Jet.pt,8,clip=True)))))
        output['pre-cuts_M3'].fill(dataset=dataset,mass=m3_recon(events[ak.num(events.Jet)>=3].Jet))
        output['post-cuts_M3'].fill(dataset=dataset,mass=m3_recon(f_events.Jet))
        output['pre-cuts_pt'].fill(dataset=dataset,pt=ak.flatten(events.Jet.pt))
        output['post-cuts_pt'].fill(dataset=dataset,pt=ak.flatten(f_events.Jet.pt))
        output['jet_pt']+=jet_pt
        output['jet_mass']+=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(f_events.Jet.mass,8,clip=True))))
        output['jet_phi']+=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(f_events.Jet.phi,8,clip=True))))
        output['jet_eta']+=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(f_events.Jet.eta,8,clip=True))))
        output['genjet_id']+=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(f_events.GenJet.partonFlavour,8,clip=True))))
        output['pdgId']+=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(f_events.GenPart.pdgId,8,clip=True))))
        return output

    def postprocess(self, accumulator):
        return accumulator

In [3]:
#Run coffea processor again, this time using the filesetData list
fileset = {
    'tt-bar': [files]
}
outputData = processor.run_uproot_job(
    fileset,
    "Events",
    JetPreProcess(),
    processor.iterative_executor,
    executor_args={"schema": HackSchema,'workers': 8}, 
    chunksize=100000,
)


Preprocessing:   0%|          | 0/1 [00:00<?, ?file/s]

Processing:   0%|          | 0/17 [00:00<?, ?chunk/s]

C:\Users\zshel\anaconda3\envs\top_tag1\lib\site-packages\awkward1\_connect\_numpy.py:163: RuntimeWarning: invalid value encountered in sqrt
  *[nplike.asarray(x) for x in inputs], **kwargs


In [19]:
from coffea.nanoevents.methods import vector
ak.behavior.update(vector.behavior)
jets = ak.zip({"pt":outputData["jet_pt"].value,"eta":outputData["jet_eta"].value,"phi":outputData["jet_phi"].value,"mass":outputData["jet_mass"].value}, with_name="PtEtaPhiMLorentzVector")
assert np.allclose(np.array(jets.x), np.array(jets.x))
assert np.allclose(np.array(jets.y), np.array(jets.y))
assert np.allclose(np.array(jets.z), np.array(jets.z))
jet

In [26]:
l=[]
n=1
for j in jets[n]:
    #j1 is leading jet(highes p_T)
    j1=jets[n][0]
    l.append(j1.delta_r(j))
pprint(l)

[0.0,
 2.3515918453827447,
 2.717351562703073,
 2.272541318487382,
 2.9244709044550623,
 4.977472614084221,
 3.342129427559331,
 1.8880814553536411]


In [27]:
from particle import Particle
l=[]
for ev in outputData['pdgId'].value[1]:
    if ev!=0:
        l.append([ev,Particle.from_pdgid(ev)])
    else:
        l.append([0,0])
pprint(l)

[[21.0, <Particle: name="g", pdgid=21, mass=0.0 MeV>],
 [21.0, <Particle: name="g", pdgid=21, mass=0.0 MeV>],
 [6.0, <Particle: name="t", pdgid=6, mass=172400 ± 700 MeV>],
 [-6.0, <Particle: name="t~", pdgid=-6, mass=172400 ± 700 MeV>],
 [21.0, <Particle: name="g", pdgid=21, mass=0.0 MeV>],
 [21.0, <Particle: name="g", pdgid=21, mass=0.0 MeV>],
 [21.0, <Particle: name="g", pdgid=21, mass=0.0 MeV>],
 [6.0, <Particle: name="t", pdgid=6, mass=172400 ± 700 MeV>]]


In [6]:
#Trying to scale existing data
events =NanoEventsFactory.from_root(files,schemaclass=HackSchema).events()


<PtEtaPhiMCollectionArray [[{eta: 0.427, ... pt: 14.7}]] type='1734354 * var * s...'>

In [13]:
events.GenJet.partonFlavour[abs(events.GenJet.partonFlavour)==6]

<Array [[], [], [], [], ... [], [], [], []] type='1734354 * var * int32[paramete...'>

In [15]:
val =abs(events.GenJet.partonFlavour)==6

In [23]:
test=(ak.cartesian(val,axis=1))

RecursionError: maximum recursion depth exceeded while calling a Python object